In [2]:
import pandas as pd
import numpy as np
import zipfile
import os
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [3]:
# Function to read CSV from ZIP archive directly
def read_csv_from_zip(zip_filepath, csv_filename):
    with zipfile.ZipFile(zip_filepath, 'r') as z:
        with z.open(csv_filename) as f:
            df = pd.read_csv(f)
    return df


In [4]:
# Paths to the ZIP files (adjust the paths according to your directory structure)
base_path = "../data/NSW/"
temperature_zip_path = os.path.join(base_path, "temperature_nsw.csv.zip")
demand_zip_path = os.path.join(base_path, "totaldemand_nsw.csv.zip")
forecastdemand_zip_paths = [os.path.join(base_path, f"forecastdemand/forecastdemand_part{i}.csv.zip") for i in range(1, 6)]

In [5]:
# Reading temperature and demand data
df_temperature_nsw = read_csv_from_zip(temperature_zip_path, "temperature_nsw.csv")
df_demand_nsw = read_csv_from_zip(demand_zip_path, "totaldemand_nsw.csv")

In [8]:
# Reading and combining forecast demand data
df_forecastdemand_nsw_parts = [read_csv_from_zip(zip_path, f"forecastdemand_part{i}.csv") for i, zip_path in enumerate(forecastdemand_zip_paths, start=1)]
df_forecastdemand_nsw = pd.concat(data_forecastdemand_nsw_parts, ignore_index=True)

NameError: name 'data_forecastdemand_nsw_parts' is not defined

In [ ]:
df_demand_nsw['DATETIME'] = pd.to_datetime(df_temperature_nsw['DATETIME'])

In [ ]:

df_temperature_nsw['DATETIME'] = pd.to_datetime(df_temperature_nsw['DATETIME'])

In [ ]:
df_temperature_nsw['Hour_Min'] = df_temperature_nsw['DATETIME'].dt.strftime('%H:%M')

In [ ]:
df_filtered = df_temperature_nsw[
    (df_temperature_nsw['DATETIME'].dt.minute <= 5) |
    (df_temperature_nsw['DATETIME'].dt.minute >= 55) |
    ((df_temperature_nsw['DATETIME'].dt.minute >= 25) & (df_temperature_nsw['DATETIME'].dt.minute <= 35))
].copy()

In [ ]:
def align_time(hour_min):
    hour, minute = map(int, hour_min.split(':'))
    if minute >= 55 or minute <= 5:
        new_minute = '00'
    elif 25 <= minute <= 35:
        new_minute = '30'
    # Reconstruct the time string; no change to hour as we don't cross hour boundaries with given ranges
    return f"{hour:02d}:{new_minute}"

In [ ]:
df_filtered['Aligned_Time'] = df_filtered['Hour_Min'].apply(align_time)

In [ ]:
df_filtered['Aligned_DATETIME'] = pd.to_datetime(df_filtered['DATETIME'].dt.date.astype(str) + ' ' + df_filtered['Aligned_Time'])


In [ ]:
# Step 2: Group by the Aligned Datetime and Calculate Mean Temperature
df_aligned_temperature_nsw = df_filtered.groupby('Aligned_DATETIME')['TEMPERATURE'].mean().reset_index(name='Mean_TEMPERATURE')


In [ ]:
# Display the head of the aligned DataFrame
print(df_aligned_temperature_nsw.head())

In [ ]:
df_aligned_temperature_nsw.rename(columns={'Aligned_DATETIME': 'DATETIME'}, inplace=True)

In [7]:
df_aligned_temperature_nsw.rename(columns={'Mean_TEMPERATURE': 'TEMPERATURE'}, inplace=True)

NameError: name 'df_aligned_temperature_nsw' is not defined

In [85]:
df_nsw = pd.merge(df_demand_nsw, df_aligned_temperature_nsw, on='DATETIME', how='inner')

In [87]:
initial_train_size = len(df_nsw) - 48


In [88]:
forecasts = np.zeros(48)
actuals = np.zeros(48)

In [89]:
for i in range(48):  # 48 observations for the 24-hour period
    train_set = df_nsw.iloc[:initial_train_size + i]
    model = SARIMAX(train_set['TOTALDEMAND'],
                    exog=train_set[['TEMPERATURE']],
                    order=(1, 1, 1),  # Example order, adjust based on your data
                    seasonal_order=(1, 1, 1, 24),  # Example seasonal order, adjust as needed
                    enforce_stationarity=False,
                    enforce_invertibility=False)
    results = model.fit(disp=0)
    
    next_temp = df_nsw.iloc[initial_train_size + i][['TEMPERATURE']]
    forecast = results.forecast(steps=1, exog=[next_temp])
    
    forecasts[i] = forecast
    actuals[i] = df_nsw.iloc[initial_train_size + i]['TOTALDEMAND']

KeyError: "None of [Index(['TEMPERATURE'], dtype='object')] are in the [columns]"

In [90]:
print(df_nsw.columns)

Index(['DATETIME', 'TOTALDEMAND', 'REGIONID', 'Mean_TEMPERATURE'], dtype='object')


In [93]:
df_nsw.dtypes

DATETIME            datetime64[ns]
TOTALDEMAND                float64
REGIONID                    object
Mean_TEMPERATURE           float64
dtype: object

In [92]:
next_temp = df_nsw.iloc[initial_train_size + i][['TEMPERATURE']]


KeyError: "None of [Index(['TEMPERATURE'], dtype='object')] are in the [index]"